In [66]:
%reset -f

In [67]:
#!/usr/bin/env python3
import pandas as pd
from tqdm import tqdm
from PIL import Image
from datetime import datetime
import torch
from torchvision import transforms
from torch import nn
from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAccuracy
from torcheval.metrics.functional import binary_accuracy
torch.manual_seed(18)
torch.cuda.is_available()

True

In [68]:
image_transform = transforms.Compose([transforms.ToTensor()])

In [69]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_file_path):
        self.data = pd.read_csv(data_file_path)
        self.image_transform = transforms.Compose([transforms.ToTensor()])


    def __getitem__(self, idx):
        image_path = self.data['path'][idx]
        image = Image.open(image_path)
        label = self.data['label'][idx]
        return self.image_transform(image), label
    

    def __len__(self):
        return len(self.data)

In [70]:
def get_test_acc(model, test_loader, device='cuda'):
    model.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    with torch.no_grad():
        test_acc = 0.0
        model.eval()
        for batch in tqdm(test_loader):
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            metric.update(outputs.flatten(), targets)
            test_acc += metric.compute().item()
    return test_acc / test_loader.__len__()

In [71]:
class FaceNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(4, 4), stride=1, padding=0)
        self.layer2 = nn.MaxPool2d(kernel_size=(3, 3), stride=1, padding=0)
        self.layer3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), stride=1, padding=0)
        self.layer4 = nn.MaxPool2d(kernel_size=(4, 4), stride=1, padding=0)
        self.layer5 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(6, 6), stride=1, padding=0)
        self.layer6 = nn.MaxPool2d(kernel_size=(5, 5), stride=1, padding=0)
        self.layer7 = nn.Conv2d(in_channels=32, out_channels=1, kernel_size=(7, 7), stride=1, padding=0)
        self.layer8 = nn.Linear(in_features=52441, out_features=32)
        self.layer9 = nn.Linear(in_features=32, out_features=16)
        self.layer10 = nn.Linear(in_features=16, out_features=8)
        self.layer11 = nn.Linear(in_features=8, out_features=1)

        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        
        x = self.layer3(x)
        x = self.relu(x)
        x = self.layer4(x)

        x = self.layer5(x)
        x = self.relu(x)
        x = self.layer6(x)

        x = self.layer7(x)
        x = self.relu(x)

        x = self.flatten(x)

        x = self.layer8(x)
        x = self.relu(x)

        x = self.layer9(x)
        x = self.relu(x)

        x = self.layer10(x)
        x = self.relu(x)

        x = self.layer11(x)
        x = self.sigmoid(x)

        return x

In [72]:
def epoch_system_out_string(epoch:int, train_loss:float, train_acc:float, val_loss:float, val_acc:float, train_loader, val_loader)->str:
    return (f'Epoch: {epoch} -- train Loss: {round(train_loss / train_loader.__len__(), 4)} \t valid Loss: {round(val_loss / val_loader.__len__(), 4)} \t train acc.:{round(train_acc / train_loader.__len__(), 4)} \t val acc.:{round(val_acc / val_loader.__len__(), 4)}')

In [73]:
def train(model, optimizer, train_loader, val_loader, epochs, loss_fn, device='cuda'):
    model.to(device)
    loss_fn.to(device)
    metric = BinaryAccuracy()
    metric.to(device)
    for epoch in (range(epochs)):
        # train the model on the training set
        train_loss = 0.0
        train_acc = 0.0
        model.train()
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            inputs, targets = batch
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs.flatten().float(), targets.float())
            loss.backward()
            optimizer.step()
            metric.update(outputs.flatten(), targets)
            train_loss += loss.item()
            train_acc += metric.compute().item()

        # evaluate the model on the validation set
        with torch.no_grad():
            val_loss = 0.0
            val_acc = 0.0
            model.eval()
            for batch in tqdm(val_loader):
                optimizer.zero_grad()
                inputs, targets = batch
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs.flatten().float(), targets.float())
                metric.update(outputs.flatten(), targets)
                val_loss += loss.item()
                val_acc += metric.compute().item()

        print(epoch_system_out_string(epoch, train_loss, train_acc, val_loss, val_acc, train_loader, val_loader))

In [74]:
batch_size = 128

training_set = MyDataset('datasets/train.csv')
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True)

valid_set = MyDataset('datasets/valid.csv')
valid_loader = DataLoader(dataset=valid_set, batch_size=batch_size, shuffle=True)

test_set = MyDataset('datasets/test.csv')
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [75]:
model = FaceNet()

In [76]:
model.forward(torch.randn(1, 3, 256, 256)), model.forward(torch.randn(1, 3, 256, 256)).size()

(tensor([[0.4829]], grad_fn=<SigmoidBackward0>), torch.Size([1, 1]))

In [77]:
loss_fn = nn.BCELoss()

params = model.parameters()
learning_rate = 3e-4
optimizer = torch.optim.Adam(params, lr=learning_rate)

# train the model
train(model=model, optimizer=optimizer, train_loader=train_loader, val_loader=valid_loader, loss_fn=loss_fn, epochs=15)

100%|██████████| 157/157 [00:45<00:00,  3.45it/s]


Epoch: 0 -- train Loss: 0.6198 	 valid Loss: 0.5583 	 train acc.:0.5978 	 val acc.:0.6559


100%|██████████| 157/157 [00:44<00:00,  3.56it/s]


Epoch: 1 -- train Loss: 0.5202 	 valid Loss: 0.4866 	 train acc.:0.6797 	 val acc.:0.7003


100%|██████████| 157/157 [00:44<00:00,  3.55it/s]


...

Epoch: 12 -- train Loss: 0.1844 	 valid Loss: 0.2683 	 train acc.:0.8434 	 val acc.:0.8463


100%|██████████| 157/157 [00:45<00:00,  3.48it/s]


Epoch: 13 -- train Loss: 0.1726 	 valid Loss: 0.2221 	 train acc.:0.8492 	 val acc.:0.8521


100%|██████████| 157/157 [00:45<00:00,  3.48it/s]

Epoch: 14 -- train Loss: 0.1628 	 valid Loss: 0.211 	 train acc.:0.8548 	 val acc.:0.8574


In [78]:
get_test_acc(model, test_loader)

100%|██████████| 157/157 [00:44<00:00,  3.49it/s]


0.9147980604202125

In [79]:
now = datetime.now()
date_string = now.strftime("%Y-%m-%d-%H-%M-%S")
model_path = f"models/face_net-{date_string}.pth"
torch.save(model.state_dict(), model_path)